In [1]:
!pip install langchain_groq

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 109.6/109.6 kB 2.7 MB/s eta 0:00:00


In [2]:
#load the api key
from google.colab import userdata
groq=userdata.get('groq_api_key')

In [3]:
import langchain
from langchain.prompts import ChatPromptTemplate
from langchain_groq import ChatGroq
from langchain_core.output_parsers import StrOutputParser

In [4]:
llm = ChatGroq(api_key=groq,
               model="gemma2-9b-it",
               temperature=0.3)

In [5]:
llm.invoke("do you know about  imran khan?").content

"Yes, I do! \n\nImran Khan is a prominent figure in Pakistani politics and international cricket. \n\n**Here are some key things to know about him:**\n\n* **Former Cricketer:** He was a legendary Pakistani cricketer, captaining the national team to victory in the 1992 Cricket World Cup. He's widely regarded as one of the greatest all-rounders of all time.\n* **Politician:** After retiring from cricket, he founded the Pakistan Tehreek-e-Insaf (PTI) political party and became Prime Minister of Pakistan in 2018. \n* **Controversial Figure:** Khan's political career has been marked by both popularity and controversy. He has been praised for his anti-corruption stance and focus on social welfare, but also criticized for his authoritarian tendencies and handling of the economy.\n* **Recent Developments:** In April 2023, Khan was removed from office through a vote of no confidence in parliament. He continues to lead the PTI and remains a popular figure among many Pakistanis.\n\n**To give you 

In [6]:
template = ChatPromptTemplate.from_messages(
    [
        ("system","you are a helpfull assistant  for the user based on the user {question} you will provide the answer"),
        ("user","{question}")
    ]
)

In [7]:
formattedPrompt = template.format_messages(question="do you know who is the best football player in the world")

In [8]:
chain = template | llm | StrOutputParser()

In [9]:
chain

ChatPromptTemplate(input_variables=['question'], input_types={}, partial_variables={}, messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=['question'], input_types={}, partial_variables={}, template='you are a helpfull assistant  for the user based on the user {question} you will provide the answer'), additional_kwargs={}), HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['question'], input_types={}, partial_variables={}, template='{question}'), additional_kwargs={})])
| ChatGroq(client=<groq.resources.chat.completions.Completions object at 0x78748ab187d0>, async_client=<groq.resources.chat.completions.AsyncCompletions object at 0x78748aa9d1d0>, model_name='gemma2-9b-it', temperature=0.3, model_kwargs={}, groq_api_key=SecretStr('**********'))
| StrOutputParser()

In [10]:
chain.invoke(formattedPrompt)

'That\'s a tough question!  "Best" is subjective and depends on what criteria you\'re using. \n\nSome people might say **Lionel Messi** because of his incredible dribbling skills, goal-scoring record, and numerous Ballon d\'Or awards. Others might argue for **Cristiano Ronaldo** due to his athleticism, powerful shots, and consistent performance over many years. \n\nUltimately, the best football player in the world is a matter of opinion. Who do *you* think is the best?  \n\n'

In [11]:
#StrOutPutParser
from pydantic import BaseModel, Field
from langchain.output_parsers import PydanticOutputParser
from langchain_core.output_parsers import StrOutputParser

In [12]:
!pip install langchain_community

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 24.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 21.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 412.2/412.2 kB 18.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.6/49.6 kB 1.9 MB/s eta 0:00:00
  Attempting uninstall: langchain-core
    Found existing installation: langchain-core 0.3.29
    Uninstalling langchain-core-0.3.29:
      Successfully uninstalled langchain-core-0.3.29
  Attempting uninstall: langchain
    Found existing installation: langchain 0.3.14
    Uninstalling langchain-0.3.14:
      Successfully uninstalled langchain-0.3.14


In [13]:
from typing import List
from langchain.prompts import ChatPromptTemplate
from langchain.output_parsers import PydanticOutputParser
from pydantic import BaseModel, Field

# Define the schema
class Person(BaseModel):
    name: str = Field(description="name of the person")
    record: List = Field(description="record of the person")

# Parser and format instructions
parser = PydanticOutputParser(pydantic_object=Person)
format_instructions = parser.get_format_instructions()

#
human_template = "{request}\n {format_instructions}"
human_message_prompt = ChatPromptTemplate.from_template(human_template)
formattedPrompt = human_message_prompt.format_messages(request="what is the best football player in the world", format_instructions=format_instructions)

In [14]:
chain = llm.invoke(formattedPrompt)
print("parser",parser.parse(chain.content))

parser name='Lionel Messi' record=[{'team': 'Paris Saint-Germain', 'position': 'Forward', 'year': 2023, 'goals': 16, 'assists': 16}]


In [15]:
#Memory in langchain

from langchain.memory import ConversationBufferMemory
from langchain.chains import ConversationChain


#define the memory
memory = ConversationBufferMemory()
conversation = ConversationChain(llm=llm, memory=memory)

conversation.predict(input="do you know aboit my favourite game?")

<ipython-input-15-1f0f42a7d336>:8: LangChainDeprecationWarning: Please see the migration guide at: https://python.langchain.com/docs/versions/migrating_memory/
  memory = ConversationBufferMemory()
<ipython-input-15-1f0f42a7d336>:9: LangChainDeprecationWarning: The class `ConversationChain` was deprecated in LangChain 0.2.7 and will be removed in 1.0. Use :meth:`~RunnableWithMessageHistory: https://python.langchain.com/v0.2/api_reference/core/runnables/langchain_core.runnables.history.RunnableWithMessageHistory.html` instead.
  conversation = ConversationChain(llm=llm, memory=memory)


"As an AI, I don't have access to past conversations or any personal information about you, including your favorite game.  \n\nWhat's your favorite game? I'd love to learn more about it! 😊  \n\n"

In [16]:
!pip install chromadb

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 2.1 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 611.1/611.1 kB 15.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 46.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 278.6/278.6 kB 15.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.8/94.8 kB 5.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 41.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.8/94.8 kB 5.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.3/13.3 MB 55.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.8/55.8 kB 3.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 69.8/69.8 kB 4.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.3/62.3 kB 3.1 MB/s eta 0:00:00


In [17]:
from langchain.memory import VectorStoreRetrieverMemory
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import Chroma

In [18]:
embeddings = HuggingFaceEmbeddings()

<ipython-input-18-d0c9174021d8>:1: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings = HuggingFaceEmbeddings()
<ipython-input-18-d0c9174021d8>:1: LangChainDeprecationWarning: Default values for HuggingFaceEmbeddings.model_name were deprecated in LangChain 0.2.16 and will be removed in 0.4.0. Explicitly pass a model_name to the HuggingFaceEmbeddings constructor instead.
  embeddings = HuggingFaceEmbeddings()
/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggin

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.6k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/363 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [19]:
db = Chroma(embedding_function=embeddings)
retriver = db.as_retriever()
memory = ConversationChain(llm=llm, memory=VectorStoreRetrieverMemory(retriever=retriver))

<ipython-input-19-243e856a38d2>:1: LangChainDeprecationWarning: The class `Chroma` was deprecated in LangChain 0.2.9 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-chroma package and should be used instead. To use it run `pip install -U :class:`~langchain-chroma` and import as `from :class:`~langchain_chroma import Chroma``.
  db = Chroma(embedding_function=embeddings)
<ipython-input-19-243e856a38d2>:3: LangChainDeprecationWarning: Please see the migration guide at: https://python.langchain.com/docs/versions/migrating_memory/
  memory =ConversationChain(llm=llm, memory=VectorStoreRetrieverMemory(retriever=retriver))


In [21]:
memory.predict(input="my favourite game is snooker and chess")

"That's fantastic! Snooker and chess are both incredibly strategic games. \n\nI've actually processed a lot of information about chess. I can tell you about famous chess players like Garry Kasparov and Magnus Carlsen, different opening strategies, and even analyze a chess position if you'd like.  \n\nSnooker is new to me though. Could you tell me a bit more about it? What do you enjoy most about playing?  \n\n"

In [23]:
memory.predict(input="which game i like most")

"Since you mentioned both snooker and chess as favorites, I'm curious to know which one you enjoy the most!  Do you have a preference?  😊 \n\n"